# filling gaps using GapCloser

This is to fill gaps on the scaffolds produced by LINKS and SLR

In [3]:
module load SOAPdenovo/2.04
module list

Currently Loaded Modulefiles:
  1) powerPlant/core    4) git/2.21.0         7) asub/2.1
  2) texlive/20151117   5) perlbrew/0.76      8) SOAPdenovo/2.04
  3) pandoc/1.19.2      6) perl/5.28.0


In [4]:
GapCloser -h

GapCloser: invalid option -- 'h'
Version:
	1.12

Contact:
	soap@genomics.org.cn

Usage:
	GapCloser [options]
	-a	<string>	input scaffold file name, required.
	-b	<string>	input library info file name, required.
	-o	<string>	output file name, required.
	-l	<int>		maximum read length (<=155), default=100.
	-p	<int>		overlap param(<=31), default=25.
	-t	<int>		thread number, default=1.
	-h	-?		output help information.



: 1

## 1. trimming

10x R1 has 16nt index at 5', this must be trimmed before gap filling. R2 does not need trimming.

In [3]:
mkdir 005.GapFilling

In [5]:
WORKDIR=005.GapFilling

In [6]:
cat $WORKDIR/readFiles_R1.txt

/input/genomic/plant/Gillenia/trifoliata/2019_Novogene_10X/customer-6g2hQKfL/BDHX190030841-1a/G3-2-AK3717_S1_L001_R1_001.fastq.gz
/input/genomic/plant/Gillenia/trifoliata/2019_Novogene_10X/customer-6g2hQKfL/BDHX190030841-1a/G3-2-AK3718_S2_L001_R1_001.fastq.gz
/input/genomic/plant/Gillenia/trifoliata/2019_Novogene_10X/customer-6g2hQKfL/BDHX190030841-1a/G3-2-AK3719_S3_L001_R1_001.fastq.gz
/input/genomic/plant/Gillenia/trifoliata/2019_Novogene_10X/customer-6g2hQKfL/BDHX190030841-1a/G3-2-AK3720_S4_L001_R1_001.fastq.gz


In [7]:
cat $WORKDIR/readFiles.txt

/input/genomic/plant/Gillenia/trifoliata/2019_Novogene_10X/customer-6g2hQKfL/BDHX190030841-1a/G3-2-AK3717_S1_L001_R1_001.fastq.gz
/input/genomic/plant/Gillenia/trifoliata/2019_Novogene_10X/customer-6g2hQKfL/BDHX190030841-1a/G3-2-AK3717_S1_L001_R2_001.fastq.gz
/input/genomic/plant/Gillenia/trifoliata/2019_Novogene_10X/customer-6g2hQKfL/BDHX190030841-1a/G3-2-AK3718_S2_L001_R1_001.fastq.gz
/input/genomic/plant/Gillenia/trifoliata/2019_Novogene_10X/customer-6g2hQKfL/BDHX190030841-1a/G3-2-AK3718_S2_L001_R2_001.fastq.gz
/input/genomic/plant/Gillenia/trifoliata/2019_Novogene_10X/customer-6g2hQKfL/BDHX190030841-1a/G3-2-AK3719_S3_L001_R1_001.fastq.gz
/input/genomic/plant/Gillenia/trifoliata/2019_Novogene_10X/customer-6g2hQKfL/BDHX190030841-1a/G3-2-AK3719_S3_L001_R2_001.fastq.gz
/input/genomic/plant/Gillenia/trifoliata/2019_Novogene_10X/customer-6g2hQKfL/BDHX190030841-1a/G3-2-AK3720_S4_L001_R1_001.fastq.gz
/input/genomic/plant/Gillenia/trifoliata/2019_Novogene_10X/customer-6g2hQKfL/BDHX190030841

In [9]:
perl /workspace/hraczw/github/programs/prinseq-lite-0.20.4/prinseq-lite.pl -h

Usage:
    perl prinseq-lite.pl [-h] [-help] [-version] [-man] [-verbose] [-fastq
    input_fastq_file] [-fasta input_fasta_file] [-fastq2 input_fastq_file_2]
    [-fasta2 input_fasta_file_2] [-qual input_quality_file] [-min_len
    int_value] [-max_len int_value] [-range_len ranges] [-min_gc int_value]
    [-max_gc int_value] [-range_gc ranges] [-min_qual_score int_value]
    [-max_qual_score int_value] [-min_qual_mean int_value] [-max_qual_mean
    int_value] [-ns_max_p int_value] [-ns_max_n int_value] [-noniupac]
    [-seq_num int_value] [-derep int_value] [-derep_min int_value]
    [-lc_method method_name] [-lc_threshold int_value] [-trim_to_len
    int_value] [-trim_left int_value] [-trim_right int_value] [-trim_left_p
    int_value] [-trim_right_p int_value] [-trim_ns_left int_value]
    [-trim_ns_right int_value] [-trim_tail_left int_value] [-trim_tail_right
    int_value] [-trim_qual_left int_value] [-trim_qual_right int_value]
    [-trim_qual_type type] [-trim_qual_rule rule] 

: 1

In [11]:
python << EOF

import sys, os

f = open('$WORKDIR/readFiles_R1.txt', 'r')

for line in f:
    sampleName = line.split('/')[-1].split('.fastq.gz')[0]
    os.system('bsub -J preseq \
               -o $WORKDIR/' + sampleName + '_preseq.out \
               -e $WORKDIR/' + sampleName + '_preseq.err \
               bsub -J trim \
               -o $WORKDIR/trim_' + sampleName + '.out \
               -e $WORKDIR/trim_' + sampleName + '.err \
               "gzip -dc ' + line.strip() + ' | \
               perl /workspace/hraczw/github/programs/prinseq-lite-0.20.4/prinseq-lite.pl \
               -verbose \
               -fastq stdin \
               -out_good stdout \
               -trim_left 16 | \
               gzip > $WORKDIR/' + sampleName + '.fastq.gz"')

f.close()

EOF

Job <746222> is submitted to default queue <lowpriority>.
Job <746223> is submitted to default queue <lowpriority>.
Job <746224> is submitted to default queue <lowpriority>.
Job <746225> is submitted to default queue <lowpriority>.


In [8]:
ll $WORKDIR

total 39566424
-rw-rw-r--. 1 hraczw powerplant 7665434413 Jun  8 15:51 G3-2-AK3717_S1_L001_R1_001.fastq.gz
-rw-rw-r--. 1 hraczw powerplant          0 Jun  8 14:28 G3-2-AK3717_S1_L001_R1_001_preseq.err
-rw-rw-r--. 1 hraczw powerplant       1502 Jun  8 14:28 G3-2-AK3717_S1_L001_R1_001_preseq.out
-rw-rw-r--. 1 hraczw powerplant 6527837345 Jun  8 15:40 G3-2-AK3718_S2_L001_R1_001.fastq.gz
-rw-rw-r--. 1 hraczw powerplant          0 Jun  8 14:28 G3-2-AK3718_S2_L001_R1_001_preseq.err
-rw-rw-r--. 1 hraczw powerplant       1502 Jun  8 14:28 G3-2-AK3718_S2_L001_R1_001_preseq.out
-rw-rw-r--. 1 hraczw powerplant 6579911841 Jun  8 15:40 G3-2-AK3719_S3_L001_R1_001.fastq.gz
-rw-rw-r--. 1 hraczw powerplant          0 Jun  8 14:28 G3-2-AK3719_S3_L001_R1_001_preseq.err
-rw-rw-r--. 1 hraczw powerplant       1502 Jun  8 14:28 G3-2-AK3719_S3_L001_R1_001_preseq.out
-rw-rw-r--. 1 hraczw powerplant 9161491184 Jun  8 16:10 G3-2-AK3720_S4_L001_R1_001.fastq.gz
-rw-rw-r--. 1 hraczw powerplant          0 Jun  8 14:

In [16]:
# raw data quality qc

echo "http://storage.powerplant.pfr.co.nz/output/genomic/plant/Gillenia/trifoliata/02_QC/"

http://storage.powerplant.pfr.co.nz/output/genomic/plant/Gillenia/trifoliata/02_QC/


## 2. GapCloser

### 2.1 create soap config file 

In [9]:
cat $WORKDIR/soap.config

#maximal read length
max_rd_len=150
[LIB]
#average insert size
avg_ins=200
#if sequence needs to be reversed
reverse_seq=0
#in which part(s) the reads are used
asm_flags=3
#use only first 100 bps of each read
rd_len_cutoff=
#in which order the reads are used while scaffolding
rank=1
# cutoff of pair number for a reliable connection (at least 3 for short insert size)
pair_num_cutoff=3
#minimum aligned length to contigs for a reliable read location (at least 32 for short insert size)
map_len=32
#a pair of fastq file, read 1 file should always be followed by read 2 file
q1=/powerplant/workspace/hraczw/github/GA/Gillenia_genome/005.GapFilling/G3-2-AK3717_S1_L001_R1_001.fastq.gz
q2=/input/genomic/plant/Gillenia/trifoliata/2019_Novogene_10X/customer-6g2hQKfL/BDHX190030841-1a/G3-2-AK3717_S1_L001_R2_001.fastq.gz
q1=/powerplant/workspace/hraczw/github/GA/Gillenia_genome/005.GapFilling/G3-2-AK3718_S2_L001_R1_001.fastq.gz
q2=/input/genomic/plant/Gillenia/trifoliata/2019_Novogene_10X/customer-6g2h

In [10]:
GapCloser -h

GapCloser: invalid option -- 'h'
Version:
	1.12

Contact:
	soap@genomics.org.cn

Usage:
	GapCloser [options]
	-a	<string>	input scaffold file name, required.
	-b	<string>	input library info file name, required.
	-o	<string>	output file name, required.
	-l	<int>		maximum read length (<=155), default=100.
	-p	<int>		overlap param(<=31), default=25.
	-t	<int>		thread number, default=1.
	-h	-?		output help information.



: 1

In [1]:
SCAF_SLR=/powerplant/workspace/hraczw/github/GA/Gillenia_genome/004.Scaffolding_10Xcontigs/SLR/scaffold_set.fa
SCAF_LINKS_I6=/workspace/hraczw/github/GA/Gillenia_genome/004.Scaffolding_10Xcontigs/k21_t20_d5000/scaffolds_k21_t10_d30000.scaffolds.fa
SCAF_LINKS_I8=/workspace/hraczw/github/GA/Gillenia_genome/004.Scaffolding_10Xcontigs/k21_t20_d5000/scaffolds_k21_t10_d40000.scaffolds.fa

In [26]:
bsub -J gapcloser \
-m wkoppb38 \
-n 20 \
-o $WORKDIR/gapcloser.out \
-e $WORKDIR/gapcloser.err \
"GapCloser \
-a $SCAF_SLR \
-b $WORKDIR/soap.config \
-o $WORKDIR/scaffolds_slr_gapClosed.fasta \
-l 150 \
-p 25 \
-t 20"

Job <746464> is submitted to default queue <lowpriority>.


In [5]:
bsub -J gapcloser \
-m wkoppb38 \
-n 20 \
-o $WORKDIR/gapcloser_links.out \
-e $WORKDIR/gapcloser_links.err \
"GapCloser \
-a $SCAF_LINKS_I6 \
-b $WORKDIR/soap.config \
-o $WORKDIR/scaffolds_links_i6_gapClosed.fasta \
-l 150 \
-p 25 \
-t 20"

Job <749692> is submitted to default queue <lowpriority>.


In [11]:
bsub -J gapcloser \
-m wkoppb39 \
-n 40 \
-o $WORKDIR/gapcloser_links_i8.out \
-e $WORKDIR/gapcloser_links_i8.err \
"GapCloser \
-a $SCAF_LINKS_I8 \
-b $WORKDIR/soap.config \
-o $WORKDIR/scaffolds_links_i8_gapClosed.fasta \
-l 150 \
-p 25 \
-t 40"

Job <231621> is submitted to default queue <lowpriority>.


NOTE: gapcloser output contains empty seqs (length = 0 bases), need to be removed before stats calculation